In [1]:
from PIL import Image
import pytesseract
from matplotlib.pyplot import imshow
%matplotlib inline
import cv2
import numpy as np
from image_processor.toolkit import *
fname = 'parking01.jpg'

In [2]:
im = cv2.imread(fname)
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
imshow(im)


In [3]:
import train_traffic_sign_detector.config as config
from train_traffic_sign_detector.tensorflow_object_detection.model_utils import load_model,visualize_detections_on_image,run_inference_for_single_image
# from train_traffic_sign_detector.tensorflow_object_detection.model_utils import create_candidate_boxes_in_frame

sess = load_model(config.MODEL_PATH)
detections = run_inference_for_single_image(sess,im)
visualized_image = visualize_detections_on_image(sess,im)

In [4]:
candidate_boxes=create_candidate_boxes_in_frame(detections, visualized_image.shape, config.SCORE_THRESHOLD)
box_images = []
for box in candidate_boxes:
    box_images.append(im[box['ymin']:box['ymax'],box['xmin']:box['xmax']])
                         

In [5]:
box_images = []
box_images.append(cv2.cvtColor(cv2.imread('park_box01.jpg'), cv2.COLOR_BGR2RGB))
box_images.append(cv2.cvtColor(cv2.imread('park_box02.jpg'), cv2.COLOR_BGR2RGB))

In [6]:
for idx,box_image in enumerate(box_images):
    h, w, c = box_image.shape
    lines = get_hough_lines_from_image(box_image)
    if (lines is None):
        continue
    save_hough_lines_image(np.copy(box_image),lines,'hough_lines_on_box_{}.jpg'.format(idx))
    segmented = segment_by_angle_kmeans(lines)
    intersections = segmented_intersections(segmented)
    save_intersection_lines_image(np.copy(box_image),intersections,'intersection_hough_lines_on_box_{}.jpg'.format(idx))
    
    default_criteria_type = cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER
    criteria = (default_criteria_type, 10, 1.0)
    flags = cv2.KMEANS_RANDOM_CENTERS
    attempts = 10
    pts = np.array(intersections).astype(np.float32)
    # run kmeans on the coords
    _, centers = cv2.kmeans(pts, 4, None, criteria, attempts, flags)[1:]
    center_points = []
    for center in centers:
        x, y = center
        x = max(x, 0)
        y = max(y, 0)
        center_points.append([x, y])

    center_points = np.float32(center_points)
    save_kmeans_coords_on_image(np.copy(box_image),center_points,'kmeans_on_box_{}.jpg'.format(idx))

    
    frame_extrema_points = np.float32([[0, 0], [0, h], [w, 0], [w, h]])
    dist_matrix = cdist(center_points, frame_extrema_points)

    points_matched = np.array([frame_extrema_points[np.argmin(i)] for i in dist_matrix])
    M = cv2.getPerspectiveTransform(center_points, points_matched)

    gray = cv2.cvtColor(box_image, cv2.COLOR_RGB2GRAY)
    threshold_value, otsu_th_image = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    dst = cv2.warpPerspective(box_image, M, (w, h))
    save_perspective_transformed_on_image(dst,'perspective_wrapped_on_box_{}.jpg'.format(idx))

In [7]:
dst = get_valid_perspective_from_localized_sign(box_images[0])
imshow(dst)

In [8]:
import pytesseract
from pytesseract import Output


d = pytesseract.image_to_data(dst, lang='fra',output_type=Output.DICT)
n_boxes = len(d['level'])
for i in range(n_boxes):
    (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
    cv2.rectangle(dst, (x, y), (x + w, y + h), 0, 2)

imshow(dst)

In [9]:
print (d['text'])